# SED Recovery

In this section, we will discuss the process of recovering the spectral energy distribution (SED) of astronomical sources using the models generated in the previous sections. This will only work if you have a trained inference model, which can be obtained by following the inference tutorial.


For most models created by Synference using Synthesizer, information about the simulator used to generate the model grid is stored in the HDF5 file (e.g. the SPS grid, SFH and metallicity models, emission model, etc). This allows Synference to automatically reconstruct the simulator to generate SEDs for any set of parameters sampled from the posterior distribution.

First we will load in a trained inference model and the corresponding fitter object used to generate the model library. 

In [ ]:
from synference import SBI_Fitter

grid_path = "../example_models/bpass_db_v4/grid_BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_2.7_Calzetti_v3_multinode.hdf5"  # noqa: E501

fitter = SBI_Fitter.load_saved_model(
    model_file="../example_models/bpass_db_v4", grid_path=grid_path, device="cpu"
)

Now we can recreate the simulator.


In [ ]:
fitter.recreate_simulator_from_grid();

Now let's get some mock photometry to recover the SEDs for. We will load a set of mock observations created using the same simulator as the model grid.

In [ ]:
import numpy as np
from IPython.display import display

path = "../example_models/bpass_db_v4/sbi_test_data_BPASS_DenseBasis_v4_final.npz"
data = np.load(path)
X_test = data["X"]
y_test = data["y"]

Now we can use the `recover_SED` method of the inference model to recover the SEDs for our mock photometry. This method will sample from the posterior distribution and generate SEDs for each sample using the reconstructed simulator.


In [ ]:
?fitter.recover_SED

Now we will loop over the recovered SEDs and plot them.

In [ ]:
%matplotlib inline

for xi_test, yi_test in zip(X_test, y_test):
    _, _, _, _, fig = fitter.recover_SED(
        X_test=xi_test, true_parameters=yi_test, num_samples=100, save_plots=False
    )

    # jupyter notebook display
    display(fig)